In [2]:
#import argparse
#import os
#import random
#import time
#import sys
import numpy as np
import pandas as pd
#import shutil

import torch
import torch.nn as nn
#import torch.nn.parallel
import torch.optim
#iimport torch.multiprocessing as mp
import torch.utils.data
import torchvision.transforms as transforms
#import torchvision.datasets as datasets
import torchvision.models as models
from PIL import Image
import cv2


class Dataset(torch.utils.data.Dataset):
    def __init__(self, list_IDs, files, ppm, tranform):
        self.filenames = files
        self.list_IDs = list_IDs
        self.ppm = ppm
        self.transform = tranform

    def __len__(self):
        return len(self.list_IDs)

    def __getitem__(self, index):
        ID = self.list_IDs[index]
        cv_in = cv2.imread('/scratch/ab9738/pollution_img/data/'+self.filenames[ID]+'.jpg',1) # color
        pil_in = Image.fromarray(cv_in)
        X = self.transform(pil_in)
        y = self.ppm[ID]
        return X,y

def double_conv(in_channels, out_channels):
    return nn.Sequential(nn.Conv2d(in_channels, out_channels, 3, padding=1),nn.ReLU(inplace=True),nn.Conv2d(out_channels, out_channels, 3, padding=1),nn.ReLU(inplace=True))

class LeUNet(nn.Module):

    def __init__(self):
        super(LeUNet,self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)
        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear') #, align_corners=True)
        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)
        self.conv_last = nn.Conv2d(64, 1, 1)
        self.features = nn.Sequential(nn.Conv2d(1, 6, 5),nn.ReLU(inplace=True),nn.MaxPool2d(2),nn.Conv2d(6, 16, 5),nn.ReLU(inplace=True),nn.MaxPool2d(2),)
        self.avgpool = nn.AdaptiveAvgPool2d((5, 5))
        self.estimator = nn.Sequential(nn.Linear(16*5*5,120),nn.ReLU(inplace=True),nn.Linear(120, 84),nn.ReLU(inplace=True),nn.Linear(84, 1),)

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)
        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)
        x = self.dconv_down4(x)
        x = self.upsample(x)
        x = torch.cat([x, conv3], dim=1)
        x = self.dconv_up3(x)
        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)
        x = self.dconv_up2(x)
        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)
        x = self.dconv_up1(x)
        x = self.conv_last(x)
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0),16*5*5)
        x = self.estimator(x)
        return x

class LeUNetEns(nn.Module):

    def __init__(self):
        super(LeUNetEns,self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)
        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear') #, align_corners=True)
        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)
        self.conv_last = nn.Conv2d(64, 1, 1)
        self.features = nn.Sequential(nn.Conv2d(1, 6, 5),nn.ReLU(inplace=True),nn.MaxPool2d(2),nn.Conv2d(6, 16, 5),nn.ReLU(inplace=True),nn.MaxPool2d(2),)
        self.avgpool = nn.AdaptiveAvgPool2d((5, 5))
        # self.estimator = nn.Sequential(nn.Linear(16*5*5,120),nn.ReLU(inplace=True),nn.Linear(120, 84),nn.ReLU(inplace=True),nn.Linear(84, 1),)

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)
        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)
        x = self.dconv_down4(x)
        x = self.upsample(x)
        x = torch.cat([x, conv3], dim=1)
        x = self.dconv_up3(x)
        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)
        x = self.dconv_up2(x)
        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)
        x = self.dconv_up1(x)
        x = self.conv_last(x)
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0),16*5*5)
        # x = self.estimator(x)
        return x


class ResNetUNet(nn.Module):

    def __init__(self):
        super(ResNetUNet,self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)
        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear') #, align_corners=True)
        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)
        self.conv_last = nn.Conv2d(64, 1, 1)
        self.features = models.resnet50()
        # self.avgpool = nn.AdaptiveAvgPool2d((5, 5))
        self.estimator = nn.Sequential(nn.Linear(1000,120),nn.ReLU(inplace=True),nn.Linear(120, 84),nn.ReLU(inplace=True),nn.Linear(84, 1),)

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)
        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)
        x = self.dconv_down4(x)
        x = self.upsample(x)
        x = torch.cat([x, conv3], dim=1)
        x = self.dconv_up3(x)
        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)
        x = self.dconv_up2(x)
        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)
        x = self.dconv_up1(x)
        x = self.conv_last(x)
        x = torch.cat((x,x,x),1)
        x = self.features(x)
        # x = self.avgpool(x)
        # x = x.view(x.size(0),16*5*5)
        x = self.estimator(x)
        return x


class StandardNet(nn.Module):

    def __init__(self, modelname='resnet101'):
        super(StandardNet, self).__init__()
        self.model = eval("models."+modelname+"()")
        if(modelname=='inception_v3'):
            self.model = eval("models."+modelname+"(aux_logits=False)")
        self.estimator = nn.Sequential(nn.Linear(1000,120),nn.ReLU(inplace=True),nn.Linear(120,84),nn.ReLU(inplace=True),nn.Linear(84, 1),)

    def forward(self, x):
        x = self.model(x)
        x = self.estimator(x)
        return x

class EnsembleNet(nn.Module):
    def __init__(self):
        super(EnsembleNet, self).__init__()
        self.resnet50_pred = models.resnet50()
        self.vgg16_pred = models.vgg16()
        self.inceptionv3_pred = models.inception_v3(aux_logits=False)
        self.estimator = nn.Sequential(nn.Linear(3000,120),nn.ReLU(inplace=True),nn.Linear(120,84),nn.ReLU(inplace=True),nn.Linear(84,1),)

    def forward(self, x):
        a = self.resnet50_pred(x)
        b = self.vgg16_pred(x)
        c = self.inceptionv3_pred(x)
        x = torch.cat((a,b,c),1)
        x = self.estimator(x)
        return x

class EnsembleLeUNet(nn.Module):
    def __init__(self):
        super(EnsembleLeUNet, self).__init__()
        self.resnet50_pred = models.resnet50()
        self.vgg16_pred = models.vgg16()
        self.inceptionv3_pred = models.inception_v3(aux_logits=False)
        self.leunet_pred = LeUNetEns()
        # self.leunet_pred = torch.nn.DataParallel(self.leunet_pred).cuda()
        self.leunet_pred.load_state_dict(torch.load("model_haze_all.pth", map_location='cpu'),strict=False)
        self.estimator = nn.Sequential(nn.Linear(3400,120),nn.ReLU(inplace=True),nn.Linear(120,84),nn.ReLU(inplace=True),nn.Linear(84,1),)

    def forward(self, x):
        a = self.resnet50_pred(x)
        b = self.vgg16_pred(x)
        c = self.inceptionv3_pred(x)
        d = self.leunet_pred(x)
        x = torch.cat((a,b,c,d),1)
        x = self.estimator(x)
        return x

class NewReLU(nn.Module):
    """ Custom ReLU layer for baseline """
    def __init__(self, alpha, beta):
        super().__init__()
        self.alpha, self.beta = alpha, beta

    def forward(self, x):
        x = self.alpha*max(x,0) + min(0,x)*self.beta/x+1e-7
        return x

class EPAPLN(nn.Module):

    def __init__(self):
        super(EPAPLN, self).__init__()
        self.conv1 = nn.Conv2d(3,32,3)
        self.conv2 = nn.Conv2d(32,32,3)
        self.maxpool = nn.MaxPool2d(3, 2)
        self.drop1 = nn.Dropout(0.5)
        self.conv3 = nn.Conv2d(32,64,3)
        self.conv4 = nn.Conv2d(64,64,3)
        self.conv5 = nn.Conv2d(64,64,3)
        self.avgpool = nn.AvgPool2d(3, 2)
        self.drop2 = nn.Dropout(0.5)
        self.conv6 = nn.Conv2d(64,96,3)
        self.conv7 = nn.Conv2d(96,96,3)
        self.conv8 = nn.Conv2d(96,96,3)
        self.conv9 = nn.Conv2d(96,96,3)
        self.activation = nn.ReLU()
        self.adaptivepool = nn.AdaptiveAvgPool2d((2,2))
        self.estimator = nn.Sequential(nn.Linear(96*2*2,120),nn.ReLU(inplace=True),nn.Linear(120, 84),nn.ReLU(inplace=True),nn.Linear(84, 1),)

    def forward(self, x):
        x = self.conv1(x)
        x = self.activation(x)
        x = self.conv2(x)
        x = self.activation(x)
        x = self.maxpool(x)
        x = self.drop1(x)
        x = self.conv3(x)
        x = self.activation(x)
        x = self.conv4(x)
        x = self.activation(x)
        x = self.conv5(x)
        x = self.activation(x)
        x = self.avgpool(x)
        x = self.drop2(x)
        x = self.conv6(x)
        x = self.activation(x)
        x = self.conv7(x)
        x = self.activation(x)
        x = self.conv8(x)
        x = self.activation(x)
        x = self.conv9(x)
        x = self.activation(x)
        x = self.adaptivepool(x)
        x = x.view(x.size(0),96*2*2)
        x = self.estimator(x)

        return x


def train(train_loader,model,criterion,optimizer):
    model.train()
    total_loss = 0.0
    epoch_samples = 0
    for x, y in train_loader:
        epoch_samples += x.size(0)
        y = y.float()
        x = x.cuda(non_blocking=True)
        y = y.cuda(non_blocking=True)

        x_var = torch.autograd.Variable(x)
        y_var = torch.autograd.Variable(y)

        yhat = model(x_var)
        loss = criterion(yhat.squeeze(),y_var)
        total_loss += loss.data.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return (total_loss/epoch_samples)


def val(val_loader,model,criterion):
    model.eval()
    total_loss = 0.0
    epoch_samples = 0
    #with torch.no_grad():
    for x, y in val_loader:
        epoch_samples += x.size(0)
        y = y.float()
        x = x.cuda(non_blocking=True)
        y = y.cuda(non_blocking=True)

        x_var = torch.autograd.Variable(x)
        y_var = torch.autograd.Variable(y)

        yhat = model(x_var)
        loss = criterion(yhat.squeeze(),y_var)
        total_loss += loss.data.item()

    return (total_loss/epoch_samples)

def freeze_weights_leunet(model):
    model.module.dconv_down1[0].weight.requires_grad, model.module.dconv_down1[0].bias.requires_grad = False, False
    model.module.dconv_down2[0].weight.requires_grad, model.module.dconv_down2[0].bias.requires_grad = False, False
    model.module.dconv_down3[0].weight.requires_grad, model.module.dconv_down3[0].bias.requires_grad = False, False
    model.module.dconv_down4[0].weight.requires_grad, model.module.dconv_down4[0].bias.requires_grad = False, False
    model.module.dconv_up1[0].weight.requires_grad, model.module.dconv_up1[0].bias.requires_grad = False, False
    model.module.dconv_up2[0].weight.requires_grad, model.module.dconv_up2[0].bias.requires_grad = False, False
    model.module.dconv_up3[0].weight.requires_grad, model.module.dconv_up3[0].bias.requires_grad = False, False
    model.module.dconv_down1[2].weight.requires_grad, model.module.dconv_down1[2].bias.requires_grad = False, False
    model.module.dconv_down2[2].weight.requires_grad, model.module.dconv_down2[2].bias.requires_grad = False, False
    model.module.dconv_down3[2].weight.requires_grad, model.module.dconv_down3[2].bias.requires_grad = False, False
    model.module.dconv_down4[2].weight.requires_grad, model.module.dconv_down4[2].bias.requires_grad = False, False
    model.module.dconv_up1[2].weight.requires_grad, model.module.dconv_up1[2].bias.requires_grad = False, False
    model.module.dconv_up2[2].weight.requires_grad, model.module.dconv_up2[2].bias.requires_grad = False, False
    model.module.dconv_up3[2].weight.requires_grad, model.module.dconv_up3[2].bias.requires_grad = False, False
    model.module.conv_last.weight.requires_grad, model.module.conv_last.bias.requires_grad = False, False
    return model

def main():
    data = pd.read_csv('../train_data.csv')
    data_train = data.sample(frac=0.8,random_state=17)
    data_val = data.loc[~data.index.isin(data_train.index)]
    files_train = list(data_train['filename'])
    files_val = list(data_val['filename'])
    ppm_train = list(data_train['ppm'])
    ppm_val = list(data_val['ppm'])
    ids_train = [i for i in range(len(files_train))]
    ids_val = [i for i in range(len(files_val))]
    data = None
    data_train = None
    data_val = None
#     model = LeUNet()
#     model = ResNetUNet()
#     model = torch.nn.DataParallel(model).cuda()
#     model.load_state_dict(torch.load("model_haze_all.pth"),strict=False) # on GPU
#     model = StandardNet('resnet50').cuda()
#     model = StandardNet('vgg16').cuda()
    model = StandardNet('inception_v3').cuda()
#     model = EPAPLN().cuda()
#     model = EnsembleNet().cuda()  
#     model = EnsembleLeUNet().cuda()  

    criterion = nn.MSELoss().cuda()
    # optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr=1e-4)
    # model = freeze_weights_leunet(model)

    optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
    train_dataset = Dataset(ids_train, files_train, ppm_train, transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor(),transforms.Normalize(mean=[0.5231, 0.5180, 0.5115],std=[0.2014, 0.2018, 0.2100]),])) # normalize
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=12)
    val_dataset = Dataset(ids_val, files_val, ppm_val, transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor(),transforms.Normalize(mean=[0.5231, 0.5180, 0.5115],std=[0.2014, 0.2018, 0.2100]),]))
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=12)

    best_loss = 1e5
    for epoch in range(500):
        train_loss = train(train_loader,model,criterion,optimizer)
        val_loss = val(val_loader,model,criterion)
        print('Epoch: %d, MSE train set: %.8f' % (epoch+1, train_loss))
        print('Epoch: %d, MSE val set: %.8f\n' % (epoch+1, val_loss))
        if val_loss < best_loss:
#             torch.save(model.state_dict(),'inception_pm_train.pth')
            best_loss = val_loss

if __name__ == "__main__":
    main()




Epoch: 1, MSE train set: 238.35291101
Epoch: 1, MSE val set: 60.75229647

Epoch: 2, MSE train set: 59.31247464
Epoch: 2, MSE val set: 60.96925855

Epoch: 3, MSE train set: 53.23261004
Epoch: 3, MSE val set: 45.69281696

Epoch: 4, MSE train set: 44.81066361
Epoch: 4, MSE val set: 48.47416370

Epoch: 5, MSE train set: 41.08159618
Epoch: 5, MSE val set: 31.94012005

Epoch: 6, MSE train set: 35.34986325
Epoch: 6, MSE val set: 27.98583209

Epoch: 7, MSE train set: 31.34575162
Epoch: 7, MSE val set: 39.98102910

Epoch: 8, MSE train set: 29.64697702
Epoch: 8, MSE val set: 31.17758730

Epoch: 9, MSE train set: 33.57702252
Epoch: 9, MSE val set: 33.32468777

Epoch: 10, MSE train set: 26.59611646
Epoch: 10, MSE val set: 32.38636110

Epoch: 11, MSE train set: 26.04520655
Epoch: 11, MSE val set: 28.27825442

Epoch: 12, MSE train set: 25.80356927
Epoch: 12, MSE val set: 25.11345245

Epoch: 13, MSE train set: 23.67475953
Epoch: 13, MSE val set: 26.89255916

Epoch: 14, MSE train set: 25.85371122
Epoc

Epoch: 112, MSE train set: 4.54330032
Epoch: 112, MSE val set: 16.53217640

Epoch: 113, MSE train set: 4.74005546
Epoch: 113, MSE val set: 12.27919823

Epoch: 114, MSE train set: 3.96086187
Epoch: 114, MSE val set: 12.87999807

Epoch: 115, MSE train set: 3.15705611
Epoch: 115, MSE val set: 11.82097590

Epoch: 116, MSE train set: 3.16222354
Epoch: 116, MSE val set: 12.00419691

Epoch: 117, MSE train set: 4.48630977
Epoch: 117, MSE val set: 14.95984777

Epoch: 118, MSE train set: 4.32034513
Epoch: 118, MSE val set: 14.30288759

Epoch: 119, MSE train set: 3.13464170
Epoch: 119, MSE val set: 13.29819120

Epoch: 120, MSE train set: 3.03803002
Epoch: 120, MSE val set: 12.93511176

Epoch: 121, MSE train set: 3.35767108
Epoch: 121, MSE val set: 14.83813049

Epoch: 122, MSE train set: 2.76349279
Epoch: 122, MSE val set: 14.06466108

Epoch: 123, MSE train set: 3.33093722
Epoch: 123, MSE val set: 15.96911961

Epoch: 124, MSE train set: 5.95800071
Epoch: 124, MSE val set: 23.62949918

Epoch: 125, 

Epoch: 220, MSE train set: 1.68709895
Epoch: 220, MSE val set: 12.22335988

Epoch: 221, MSE train set: 1.57315719
Epoch: 221, MSE val set: 12.38922670

Epoch: 222, MSE train set: 1.47337117
Epoch: 222, MSE val set: 12.41218227

Epoch: 223, MSE train set: 1.31257158
Epoch: 223, MSE val set: 12.25200527

Epoch: 224, MSE train set: 1.44159400
Epoch: 224, MSE val set: 12.07151576

Epoch: 225, MSE train set: 1.72450892
Epoch: 225, MSE val set: 11.87416977

Epoch: 226, MSE train set: 1.36269301
Epoch: 226, MSE val set: 11.99248188

Epoch: 227, MSE train set: 1.30271670
Epoch: 227, MSE val set: 12.40803822

Epoch: 228, MSE train set: 1.46065744
Epoch: 228, MSE val set: 13.74462955

Epoch: 229, MSE train set: 1.48933357
Epoch: 229, MSE val set: 12.12409973

Epoch: 230, MSE train set: 1.55203712
Epoch: 230, MSE val set: 13.72858922

Epoch: 231, MSE train set: 1.68883242
Epoch: 231, MSE val set: 13.00038775

Epoch: 232, MSE train set: 1.66326323
Epoch: 232, MSE val set: 16.81239002

Epoch: 233, 

Epoch: 328, MSE train set: 3.06181904
Epoch: 328, MSE val set: 20.20415512

Epoch: 329, MSE train set: 1.90556562
Epoch: 329, MSE val set: 16.02277211

Epoch: 330, MSE train set: 1.89999463
Epoch: 330, MSE val set: 15.29828881

Epoch: 331, MSE train set: 1.24514799
Epoch: 331, MSE val set: 12.65011190

Epoch: 332, MSE train set: 1.31431834
Epoch: 332, MSE val set: 14.24237399

Epoch: 333, MSE train set: 2.33786846
Epoch: 333, MSE val set: 14.06801843

Epoch: 334, MSE train set: 1.39240911
Epoch: 334, MSE val set: 12.24224435

Epoch: 335, MSE train set: 2.04184761
Epoch: 335, MSE val set: 15.82669927

Epoch: 336, MSE train set: 4.28913910
Epoch: 336, MSE val set: 14.76965059

Epoch: 337, MSE train set: 4.31913771
Epoch: 337, MSE val set: 13.60978905

Epoch: 338, MSE train set: 2.78028796
Epoch: 338, MSE val set: 12.55720165

Epoch: 339, MSE train set: 1.97770600
Epoch: 339, MSE val set: 12.15236748

Epoch: 340, MSE train set: 1.52711705
Epoch: 340, MSE val set: 11.50886308

Epoch: 341, 

Epoch: 436, MSE train set: 0.83266302
Epoch: 436, MSE val set: 10.98626994

Epoch: 437, MSE train set: 0.77668829
Epoch: 437, MSE val set: 10.44266557

Epoch: 438, MSE train set: 0.78418027
Epoch: 438, MSE val set: 11.17964407

Epoch: 439, MSE train set: 0.82227513
Epoch: 439, MSE val set: 10.47624591

Epoch: 440, MSE train set: 0.85256259
Epoch: 440, MSE val set: 10.55746873

Epoch: 441, MSE train set: 0.71507337
Epoch: 441, MSE val set: 10.87047189

Epoch: 442, MSE train set: 0.80352469
Epoch: 442, MSE val set: 10.55810382

Epoch: 443, MSE train set: 0.71169694
Epoch: 443, MSE val set: 11.11938028

Epoch: 444, MSE train set: 1.48072204
Epoch: 444, MSE val set: 13.34807769

Epoch: 445, MSE train set: 4.78085166
Epoch: 445, MSE val set: 19.28512245

Epoch: 446, MSE train set: 6.58259744
Epoch: 446, MSE val set: 16.79131872

Epoch: 447, MSE train set: 4.16127468
Epoch: 447, MSE val set: 16.62725219

Epoch: 448, MSE train set: 2.46397215
Epoch: 448, MSE val set: 12.55817153

Epoch: 449, 